# Примеры работы с векторными представлениями слов

Для экспериментов будем использовать библиотеку [gensim.](https://radimrehurek.com/gensim/auto_examples/index.html#documentation)

Используем предобученную модель с сайта [RusVectores.](https://rusvectores.org/ru/models/)

Необходимо отметить, что в существующих универсальных библиотеках (spacy, natasha) также реализованы возможности для работы с векторными представлениями.

In [1]:
import gensim
from gensim.models import word2vec

In [2]:
model_path = 'corpus/ruscorpora_mystem_cbow_300_2_2015.bin.gz'

In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

## Выдаем ближайшие слова

In [4]:
words = ['король_S', 'королева_S', 'мужчина_S', 'женщина_S']

In [5]:
for word in words:
    if word in model:
        print('\nСЛОВО - {}'.format(word))
        print('5 ближайших соседей слова:')
        for word, sim in model.most_similar(positive=[word], topn=5):
            print('{} => {}'.format(word, sim))
    else:
        print('Слово "{}" не найдено в модели'.format(word))


СЛОВО - король_S
5 ближайших соседей слова:
принц_S => 0.672407865524292
герцог_S => 0.6092391610145569
королева_S => 0.5848619937896729
император_S => 0.5664135813713074
курфюрст_S => 0.5498137474060059

СЛОВО - королева_S
5 ближайших соседей слова:
король_S => 0.5848619937896729
принцесса_S => 0.5745126008987427
герцогиня_S => 0.5264037847518921
принц_S => 0.4866287410259247
герцог_S => 0.4809737503528595

СЛОВО - мужчина_S
5 ближайших соседей слова:
женщина_S => 0.7638137340545654
девушка_S => 0.6010492444038391
вамп_A => 0.571452260017395
девица_S => 0.5431127548217773
немолодой_A => 0.5392476320266724

СЛОВО - женщина_S
5 ближайших соседей слова:
девушка_S => 0.7725452184677124
мужчина_S => 0.7638137340545654
девица_S => 0.6532836556434631
дама_S => 0.6249092817306519
человек_S => 0.5979774594306946


## Находим близость между словами и строим аналогии

In [6]:
print(model.similarity('король_S', 'королева_S'))

0.584862


In [7]:
print(model.similarity('мужчина_S', 'женщина_S'))

0.76381373


In [8]:
print(model.most_similar(positive=['король_S', 'женщина_S'], negative=['мужчина_S']))

[('принц_S', 0.5996884107589722), ('герцог_S', 0.5775512456893921), ('королева_S', 0.5213630199432373), ('принцесса_S', 0.5140257477760315), ('герцогиня_S', 0.5112662315368652), ('император_S', 0.5007482171058655), ('посланник_S', 0.4788142442703247), ('посол_S', 0.453449547290802), ('царь_S', 0.4492655396461487), ('императрица_S', 0.4435415267944336)]


In [9]:
print(model.similarity('франция_S', 'париж_S'))

0.4549625


In [10]:
print(model.similarity('испания_S', 'мадрид_S'))

0.39409316


In [11]:
print(model.most_similar(positive=['париж_S', 'испания_S'], negative=['франция_S']))

[('лондон_S', 0.6315737366676331), ('берлин_S', 0.6051103472709656), ('мюнхен_S', 0.5779414176940918), ('нью-йорк_S', 0.5686920881271362), ('москва_S', 0.5660902261734009), ('амстердам_S', 0.5587795376777649), ('милан_S', 0.5564035773277283), ('венеция_S', 0.5518475770950317), ('авиньон_S', 0.5482670068740845), ('копенгаген_S', 0.539797842502594)]


## Обучим word2vec на наборе данных imdb

In [12]:
import re
import pandas as pd
import numpy as np
from typing import Dict, Tuple
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Paladin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
# Загрузка данных
imdb_df = pd.read_csv("data/imdb_labelled.txt", delimiter='\t', header=None, names=['text', 'value'])
imdb_df.head()

,text,value
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [14]:
# Подготовим корпус
corpus = []
stop_words = stopwords.words('english')
tok = WordPunctTokenizer()
for line in imdb_df['text'].values:
    line1 = line.strip().lower()
    line1 = re.sub("[^a-zA-Z]"," ", line1)
    text_tok = tok.tokenize(line1)
    text_tok1 = [w for w in text_tok if not w in stop_words]
    corpus.append(text_tok1)

In [15]:
corpus[:5]

[['slow',
  'moving',
  'aimless',
  'movie',
  'distressed',
  'drifting',
  'young',
  'man'],
 ['sure',
  'lost',
  'flat',
  'characters',
  'audience',
  'nearly',
  'half',
  'walked'],
 ['attempting',
  'artiness',
  'black',
  'white',
  'clever',
  'camera',
  'angles',
  'movie',
  'disappointed',
  'became',
  'even',
  'ridiculous',
  'acting',
  'poor',
  'plot',
  'lines',
  'almost',
  'non',
  'existent'],
 ['little', 'music', 'anything', 'speak'],
 ['best',
  'scene',
  'movie',
  'gerardo',
  'trying',
  'find',
  'song',
  'keeps',
  'running',
  'head']]

In [16]:
# количество текстов в корпусе не изменилось и соответствует целевому признаку
assert imdb_df.shape[0]==len(corpus)

In [17]:
%time model_imdb = word2vec.Word2Vec(corpus, workers=4, min_count=10, window=10, sample=1e-3)

Wall time: 37.9 ms


In [18]:
# Проверим, что модель обучилась
print(model_imdb.wv.most_similar(positive=['find'], topn=5))

[('ever', 0.34744006395339966), ('watching', 0.31204521656036377), ('predictable', 0.28830358386039734), ('acting', 0.2815362811088562), ('really', 0.27776145935058594)]


## Решение задачи анализа тональности текста на основе модели word2vec

In [19]:
def sentiment(v, c):
    model = Pipeline(
        [("vectorizer", v), 
         ("classifier", c)])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print_accuracy_score_for_classes(y_test, y_pred)

In [20]:
class EmbeddingVectorizer(object):
    '''
    Для текста усредним вектора входящих в него слов
    '''
    def __init__(self, model):
        self.model = model
        self.size = model.vector_size

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([np.mean(
            [self.model[w] for w in words if w in self.model] 
            or [np.zeros(self.size)], axis=0)
            for words in X])

In [21]:
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [22]:
# Обучающая и тестовая выборки
boundary = 700
X_train = corpus[:boundary] 
X_test = corpus[boundary:]
y_train = imdb_df.value.values[:boundary]
y_test = imdb_df.value.values[boundary:]

In [23]:
sentiment(EmbeddingVectorizer(model_imdb.wv), LogisticRegression(C=5.0))

Метка 	 Accuracy
0 	 0.46153846153846156
1 	 0.6
